# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 14 2023 9:02AM,264720,10,HH-42596,Hush Hush,Released
1,Jun 14 2023 9:02AM,264720,10,HH-42597,Hush Hush,Released
2,Jun 14 2023 9:02AM,264720,10,HH-42590,Hush Hush,Released
3,Jun 14 2023 9:02AM,264720,10,HH-42591,Hush Hush,Released
4,Jun 14 2023 9:02AM,264720,10,HH-42592,Hush Hush,Released
5,Jun 14 2023 9:02AM,264720,10,HH-42593,Hush Hush,Released
6,Jun 14 2023 9:02AM,264720,10,HH-42594,Hush Hush,Released
7,Jun 14 2023 9:02AM,264720,10,HH-42595,Hush Hush,Released
8,Jun 14 2023 9:02AM,264720,10,HH-42584,Hush Hush,Released
9,Jun 14 2023 9:02AM,264720,10,HH-42585,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,264716,Released,15
26,264717,Released,10
27,264718,Released,1
28,264719,Released,1
29,264720,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264716,NaN,NaN,15.0
264717,NaN,NaN,10.0
264718,NaN,NaN,1.0
264719,NaN,NaN,1.0
264720,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264654,16.0,5.0,5.0
264664,0.0,0.0,1.0
264674,0.0,1.0,5.0
264683,0.0,0.0,2.0
264684,0.0,0.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264654,16,5,5
264664,0,0,1
264674,0,1,5
264683,0,0,2
264684,0,0,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264654,16,5,5
1,264664,0,0,1
2,264674,0,1,5
3,264683,0,0,2
4,264684,0,0,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264654,16,5,5
1,264664,,,1
2,264674,,1,5
3,264683,,,2
4,264684,,,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 14 2023 9:02AM,264720,10,Hush Hush
14,Jun 14 2023 9:02AM,264715,18,Capsugel
15,Jun 14 2023 9:00AM,264719,10,"Uniderm USA, Inc"
16,Jun 14 2023 9:00AM,264718,20,"Exact-Rx, Inc."
17,Jun 14 2023 8:59AM,264717,10,"Digital Brands, LLC"
27,Jun 14 2023 8:58AM,264716,10,Alikai Health
42,Jun 14 2023 8:53AM,264714,19,"AdvaGen Pharma, Ltd"
43,Jun 14 2023 8:51AM,264713,20,"HVL, LLC dba Atrium Innovations"
45,Jun 14 2023 8:49AM,264712,19,"Methapharm, Inc."
46,Jun 14 2023 8:42AM,264711,10,"Virtue Rx, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 14 2023 9:02AM,264720,10,Hush Hush,,,14
1,Jun 14 2023 9:02AM,264715,18,Capsugel,,,1
2,Jun 14 2023 9:00AM,264719,10,"Uniderm USA, Inc",,,1
3,Jun 14 2023 9:00AM,264718,20,"Exact-Rx, Inc.",,,1
4,Jun 14 2023 8:59AM,264717,10,"Digital Brands, LLC",,,10
5,Jun 14 2023 8:58AM,264716,10,Alikai Health,,,15
6,Jun 14 2023 8:53AM,264714,19,"AdvaGen Pharma, Ltd",,,1
7,Jun 14 2023 8:51AM,264713,20,"HVL, LLC dba Atrium Innovations",,,2
8,Jun 14 2023 8:49AM,264712,19,"Methapharm, Inc.",,,1
9,Jun 14 2023 8:42AM,264711,10,"Virtue Rx, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2023 9:02AM,264720,10,Hush Hush,14,,
1,Jun 14 2023 9:02AM,264715,18,Capsugel,1,,
2,Jun 14 2023 9:00AM,264719,10,"Uniderm USA, Inc",1,,
3,Jun 14 2023 9:00AM,264718,20,"Exact-Rx, Inc.",1,,
4,Jun 14 2023 8:59AM,264717,10,"Digital Brands, LLC",10,,
5,Jun 14 2023 8:58AM,264716,10,Alikai Health,15,,
6,Jun 14 2023 8:53AM,264714,19,"AdvaGen Pharma, Ltd",1,,
7,Jun 14 2023 8:51AM,264713,20,"HVL, LLC dba Atrium Innovations",2,,
8,Jun 14 2023 8:49AM,264712,19,"Methapharm, Inc.",1,,
9,Jun 14 2023 8:42AM,264711,10,"Virtue Rx, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2023 9:02AM,264720,10,Hush Hush,14,,
1,Jun 14 2023 9:02AM,264715,18,Capsugel,1,,
2,Jun 14 2023 9:00AM,264719,10,"Uniderm USA, Inc",1,,
3,Jun 14 2023 9:00AM,264718,20,"Exact-Rx, Inc.",1,,
4,Jun 14 2023 8:59AM,264717,10,"Digital Brands, LLC",10,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2023 9:02AM,264720,10,Hush Hush,14.0,NaN,NaN
1,Jun 14 2023 9:02AM,264715,18,Capsugel,1.0,NaN,NaN
2,Jun 14 2023 9:00AM,264719,10,"Uniderm USA, Inc",1.0,NaN,NaN
3,Jun 14 2023 9:00AM,264718,20,"Exact-Rx, Inc.",1.0,NaN,NaN
4,Jun 14 2023 8:59AM,264717,10,"Digital Brands, LLC",10.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2023 9:02AM,264720,10,Hush Hush,14.0,0.0,0.0
1,Jun 14 2023 9:02AM,264715,18,Capsugel,1.0,0.0,0.0
2,Jun 14 2023 9:00AM,264719,10,"Uniderm USA, Inc",1.0,0.0,0.0
3,Jun 14 2023 9:00AM,264718,20,"Exact-Rx, Inc.",1.0,0.0,0.0
4,Jun 14 2023 8:59AM,264717,10,"Digital Brands, LLC",10.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3176408,62.0,1.0,0.0
18,529425,2.0,0.0,0.0
19,1852885,29.0,5.0,16.0
20,1058780,7.0,0.0,0.0
22,529409,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3176408,62.0,1.0,0.0
1,18,529425,2.0,0.0,0.0
2,19,1852885,29.0,5.0,16.0
3,20,1058780,7.0,0.0,0.0
4,22,529409,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,62.0,1.0,0.0
1,18,2.0,0.0,0.0
2,19,29.0,5.0,16.0
3,20,7.0,0.0,0.0
4,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,62.0
1,18,Released,2.0
2,19,Released,29.0
3,20,Released,7.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,18,19,20,22
Status,,,,,
Completed,0.0,0.0,16.0,0.0,0.0
Executing,1.0,0.0,5.0,0.0,1.0
Released,62.0,2.0,29.0,7.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,18,19,20,22
0,Completed,0.0,0.0,16.0,0.0,0.0
1,Executing,1.0,0.0,5.0,0.0,1.0
2,Released,62.0,2.0,29.0,7.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,18,19,20,22
0,Completed,0.0,0.0,16.0,0.0,0.0
1,Executing,1.0,0.0,5.0,0.0,1.0
2,Released,62.0,2.0,29.0,7.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()